# Abrieb von Besen und Kehrmaschinen
## Modellansatz
Nach einer Quantifizierung von Bertling et al. (2018) fallen in Deutschland pro Kopf und Jahr 38,3 g Mikroplastikemissionen durch den Abrieb von Besen und Kehrmaschinen an, die folgendermaßen aufgegliedert sind:
* Privater Bereich & Stadtreinigungen: $e_b = 28,3 g/(cap·a)$
* Kehrmaschinen Landwirtschaft: $e_l = 9,6 g/(cap · a)$
* Kehrmaschinen Stadtreinigungen: $e_k = 0,4 g/(cap · a)$

Wir gehen davon aus, dass der Posten "Privater Bereich & Stadtreinigungen" sich in Abgrenzung zu "Kehrmaschinen Stadtreinigung" auf Emissionen aus der manuellen Reinigung mit Besen bezieht. Die Emissionen aus der Landwirtschaft werden hier aufgrund des städtischen Bezugsraums nicht weiter berücksichtigt.

Die jährlichen Emissionsmengen $E_b$ aus Besen und $E_k$ aus Kehrmaschinen rechnen wir auf die Einwohnerzahl $p$ des Betrachtungsraums hoch:

$E_t = e_t · p \quad$ für $t \in \{b, k\}$

Zur Ermittlung der räumlichen Verteilung der Emissionen nehmen wir an, dass die jährlichen Emissionen $E_b^i$ bzw. $E_k^i$ für einen Streckenabschnitt i proportional zur Länge $l^i$ des Streckenabschnitts und zur Reinigungshäufigkeit $f_b^i$ bzw. $f_k^i$ sind:

$E_t^i = \alpha_t \cdotp E_t \cdotp l^i \cdotp f_t^i \quad$ für $t \in \{b, k\} \quad $ (1)

Durch die Randbedingungen $E_t = \sum E_t^j$ ergibt sich für die jeweilige Proportionalitätskonstante $\alpha_t$:

$\alpha_t = \frac{1}{\sum l^j f_t^j} \quad$ für $t \in \{b, k\}$ (2)

Die streckenspezifischen Emissionsmengen $ϵ_b^i$ bzw. $ϵ_k^i$ ergeben sich durch Division der Gleichung (1) durch $l^i$ und Einsetzen von (2):

$ϵ_t^i = \frac{1}{\sum l^j f_t^j} \cdotp E_t \cdotp f_t^i \quad$ für $t \in \{b, k\}$

Durch Summieren über t ergibt sich schließlich die gesamte streckenspezifische Emissionsmenge des $i$-ten Streckenabschnitts:

$ϵ^i = ϵ_b^i + ϵ_k^i = \frac{1}{\sum l^j f_b^j} \cdotp E_b \cdotp f_b^i + \frac{1}{\sum l^j f_k^j} \cdotp E_k \cdotp f_k^i$

Nicht berücksichtigt sind bislang:
* Die Breite der Fahrbahn bzw. die Anzahl der Fahrstreifen
* Die Reinigungsintensität
* Die Geometrie des Streckenabschnitts (Kurvigkeit, Längs- und Querneigung)
* Die Oberflächenbeschaffenheit der Fahrbahn bzw. des Gehwegs
* Die Witterung (Niederschlag, Wind, rel. Luftfeuchtigkeit)
* Materielle Zusammensetzung der eingesetzten Besen / Bürsten

## Datenquellen
Die Pro-Kopf-Emissionen des Abriebs werden Bertling et al. (2018) entnommen, siehe oben. Für die Bevölkerungszahl Berlins werden die Angaben des Amts für Statistik Berlin-Brandenburg verwendet.

Zur Bestimmung der Reinigungshäufigkeiten $f_t^i$ und der Längen der Streckenabschnitte $l^i$ wird am Beispiel Berlins das Straßenreinigungsverzeichnis und die dort zugeordneten Reinigungsklassen herangezogen.

### Straßenreinigungsverzeichnis und Reinigungsklassen in Berlin
* **Datenquelle:** Berliner Stadtreinigung (Berlin Open Data 2015)
* **Kurzbeschreibung:** "Auf Basis des Berliner Straßenreinigungsverzeichnisses (StrReinVerz) werden alle öffentlichen Straßen, Plätze und Wege in verschiedene Verzeichnisse unterteilt. Merkmale hierfür sind Ausbau und Verkehrslage. Dementsprechend ist festgelegt, wer die Reinigung übernimmt. Zusätzlich wird die Häufigkeit der Reinigung in Reinigungsklassen definiert" (Berlin Open Data 2015).
* **Format:** Vier verschiedene Formate (DXF, GML, SHP, MITAB und PDF) für je drei verschiedene Koordinatensysteme (Soldner, ETRS89, WGS84) sind verfügbar. Wir verwenden SHP, da dieses vom Python-Package Geopandas unterstützt wird, im Koordinatensystem ETRS89, das im Vergleich zu WGS84 für kleinräumige Sachverhalte besser geeignet ist (vgl. Open Data Düsseldorf 2020) und als modernes Koordinatensystem weiter verbreitet ist als Soldner.
* **Räumliche Abdeckung:** Berlin
* **Räumliche Auflösung:** Straßenabschnitte
* **Zeitliche Abdeckung:** Momentaufnahme (09.11.2015)
* **Zeitliche Auflösung:** Momentaufnahme (09.11.2015)
* **Aktualität:** laut Anbieter wöchentliche Aktualisierung, jedoch 2015 letztmalig aktualisiert

Über die oben beschriebenen Daten hinaus wird vom Anbieter eine Visualisierung der Daten als Webanwendung zur Verfügung gestellt, diese ist allerdings aktuell nicht funktionsfähig (Berlin Open Data, o.J.).


## Daten einlesen
### Straßenreinigungsverzeichnis und Reinigungsklassen in Berlin

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os

In [ ]:
# BEACHTE: DATEIPFAD MUSS GEGEBENENFALLS ANGEPASST WERDEN
mypath = os.getcwd()
bsr_path = os.path.join(mypath, "Eingangsdaten", "Reinigungsklassen.zip")
bsr = gpd.read_file(bsr_path)

### Bevölkerungszahl von Berlin
Die Bevölkerungszahl Berlins im Jahr 2020 betrug 3,6641 Mio. (Amt für Statistik Berlin-Brandenburg, o.J.).

In [ ]:
p_Berlin = 3.6651E6

### Pro-Kopf-Emissionen
Nach Bertling et al. (2018) (siehe oben):
* Privater Bereich & Stadtreinigungen: $e_b = 28,3 g/(cap·a)$
* Kehrmaschinen Stadtreinigungen: $e_k = 0,4 g/(cap · a)$

In [ ]:
e_b = 28.3
e_k = 0.4

## Explorative Datenanalyse
Da keine Dokumentation des Datensatzes vorliegt, interpretieren wir die Bedeutung der Attribute aufgrund ihrer Bezeichnungen und der vorkommenden Werte. Ein Blick auf die ersten Zeilen des Datensatzes, eine einfache beschreibende Statistik und die Wertebereiche für Attribute mit einer einstelligen Anzahl unterschiedlicher Ausprägungen gibt uns einen ersten Eindruck:

In [ ]:
# Erste fünf Zeilen:
bsr.head()

In [ ]:
# Beschreibende Statistik:
print ("Zeilenanzahl: " + str(len(bsr)))
bsr.describe()

In [ ]:
# Ausprägungshäufigkeiten der Attribute mit einstelliger Anzahl unterschiedlicher Ausprägungen:
for var in {'ENTOBJTYP', 'RVZRKL', 'HOF'}:
  print (var + ':')
  print (bsr[var].value_counts(dropna=False))
  print ()

In [ ]:
# Ausprägungshäufigkeiten als Histogramme:
vars = ['ENTOBJTYP', 'RVZRKL', 'HOF']
fig, ax = plt.subplots(1, 3, figsize=(25, 5))
for count, item in enumerate(ax):
  bsr[vars[count]].value_counts(dropna=False).plot(kind="bar", ax = item, title="Ausprägungshäufigkeiten des Attributs " + vars[count])

Die folgenden Erkenntnisse über die einzelnen Attribute lassen sich ableiten:
*   Der Datensatz enthält 88.155 Straßenabschnitte, die mit einer eindeutigen Kennung ('ROID') versehen sind.
*   Jeder Straßenabschnitt ist einem von vier Typen zugeordnet (ENTOBJTYP). Die Bedeutung der Typenkürzel ('EOT_GWG', 'EOT_PLABEF', 'EOT_TUNANL', 'EOT_PLAGFL') wird weiter unten analysiert. Nahezu alle Straßenabschnitte (87.486, entspricht 99,2 Prozent) sind dem Typ 'EOT_GWG' zugeordnet.
*   Jedem Straßenabschnitt ist zudem eine 'BLOCKSEITE' ("Gebiet, in dem sich bebaute Grundstücke mit Adressen zu diesem Straßenabschnitt befinden", vgl. GovData 2013) zugeordnet. Diese besteht aus einer, bis auf eine Ausnahme eindeutigen, Ziffernfolge.
*    Nahezu allen Straßenabschnitten (87.786, entspricht 99,6 Prozent) ist eine von 9 Reinigungsklassen ('RVZRKL') zugewiesen. Neben den dokumentierten und weiter unten beschriebenen Reinigungsklassen kommt der Wert 'P' vor.
*   Die Name der Straße, auf der der Straßenabschnitt liegt, wird für alle Straßenabschnitte unter 'STR_ANL' angegeben.
*   Für eine Vielzahl der Straßenabschnitte (84.253 bzw. 84.000, entspricht 95,6 Prozent bzw. 95,3 Prozent) wird zudem der Straßenname der kreuzenden oder einmündenden Straßen angegeben, an der der Abschnitt beginnt ('STR_VON') bzw. endet ('STR_BIS').
*   Für jeden Straßenabschnitt wird zudem ein zuständiger Betriebshof ('HOF') angegeben: 'R1' bis 'R5'.
*   Unter 'geometry' ist der Verlauf des Straßenabschnitts als Liste von Koordinaten im verwendeten Koordinatensystem (ETRS89) erfasst.

Für ein tiefergehendes Verständnis für die Daten visualisieren wir einen Ausschnitt ('HOF' = 'R5') in der folgenden interaktiven Karte. Die inhaltlich bedeutsamen Variablen zu einem Straßenabschnitt werden als Mouse-Over-Effekt eingeblendet.

In [ ]:
import folium 

# Karte initialisieren
m=folium.Map(location=[52.5, 13.4], zoom_start=11)

# Farbcodierung des Objekttyps
c_map = {
    'EOT_GWG': 'gray',
    'EOT_PLABEF': 'limegreen',
    'EOT_TUNANL': 'royalblue',
    'EOT_PLAGFL': 'orangered'
}

# Gestrichelte Linien für Privatstraßen
dash_map = dict.fromkeys(['C', 'A3', 'A4', 'A2b', 'A2a', 'A1a', 'A1b', 'B', None], 'none')
dash_map.update(dict.fromkeys('P', '2 8'))


style_function = lambda x: {'fillColor': '#ffffff', 
                            'color': c_map[x['properties']['ENTOBJTYP']],
                            'dashArray': dash_map[x['properties']['RVZRKL']],
                            'fillOpacity': 0.1, 
                            'weight': 2} # war auf 0.1

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#ff0000', 
                                'fillOpacity': 0.50, 
                                'weight': 1} # war auf 0.1
NIL = folium.features.GeoJson(
    #data = bsr.loc[(bsr['RVZRKL'] == 'P') | (bsr['ENTOBJTYP'] != 'EOT_GWG')],
    #data = bsr.loc[bsr['RVZRKL'] == 'P'],
    #data = bsr.loc[pd.isnull(bsr['RVZRKL'])],
    #data = bsr[0:16000],
    data = bsr.loc[bsr['HOF'] == 'R5'],
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['ENTOBJTYP', 'RVZRKL', 'STR_ANL', 'STR_VON', 'STR_BIS', 'HOF'],
        aliases=['ENTOBJTYP', 'RVZRKL', 'STR_ANL', 'STR_VON', 'STR_BIS', 'HOF'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

m

Durch eine stichprobenartige Untersuchung der Straßenabschnitte auf ihre Merkmale leiten wir folgende Annahmen her:

* Die Typen der Straßenabschnitte werden folgendermaßen codiert:
  *   Der am häufigsten vorkommende Straßentyp ('EOT_GWG') steht für gewöhnliche Straßen und Wege.
  *   Mit 'EOT_PLABEF' werden Parkplätze, Grünstreifen, sowie einige Plätze codiert.
  *   Der Typ 'EOT_PLAGFL' bezeichnet Plätze.
  *   Als 'EOT_TUNANL' werden Tunnel markiert.
* Abschnitte von Straßen und Wegen werden in der Regel mit je einem Linienzug je Straßenseite, also doppelt, erfasst.
* Flächige Objekte wie Plätze werden als geschlossener Linienzug abgebildet.
* Der Wert 'RVZRKL' = 'P' codiert Privatstraßen. Für sie ist keine Reinigungsklasse definiert.


## Daten vorverarbeiten
Für den Zweck dieses Modells ist neben der Geometrie eines Streckenabschnitts in erster Linie seine Reinigungsklasse relevant. Sie legt fest, was zu reinigen ist, wer für die Reinigung zuständig ist und wie häufig die Reinigung zu erfolgen hat. Nach (BSR, o.J.) sind für alle Reinigungsklasse zu reinigen:

* Gehweg, wenn vorhanden
* Straßenbegleitgrün
* Fahrbahn

Für die mit 'A' beginnenden Klassen ist zusätzlich der Mittelstreifen zu reinigen, was hier jedoch, ebenso wie das Straßenbegleitgrün, vereinfachend nicht gesondert betrachtet wird.

Die Zuständigkeiten und Reinigungshäufigkeiten nach (ebd.) sind in folgender Tabelle dargestellt:

| Reinigungsklasse | Zuständigkeit | Reinigungshäufigkeit (pro Woche) |
| :----------------: | :-------------: | :--------------------------------: |
| A1a | BSR | 10 |
| A1b | BSR | 7 |
| A2a | BSR | 6 |
| A2b | BSR | 5 |
| A3 | BSR | 3 |
| A4 | BSR | 1 |
| B | BSR | 1 |
| C | Anlieger | 1 |

Für die Modellierung treffen wir folgende vereinfachende Annahmen:
* Die tatsächliche Reinigungshäufigkeit entspricht der vorgeschriebenen.
* Die von der BSR gereinigten Fahrbahnen werden mit Kehrmaschinen gereinigt.
* Alle Gehwege werden mit Besen gereinigt.
* Alle Straßen verfügen über einen Fahrstreifen und einen Gehweg je Richtung.
* Privatstraßen und Straßenabschnitte ohne angegebene Reinigungsklasse werden entsprechend der Reinigungsklasse C gereinigt (1 mal pro Woche durch die Anlieger).

Parkplätze, Grünstreifen und Plätze (n = 648) werden nicht berücksichtigt.

Wir filtern im Folgenden die Daten entsprechend dieser Einschränkungen und berechnen auf Grundlage der beschriebenen Annahmen die Reinigungshäufigkeiten $f_t^i$ und der Längen der Streckenabschnitte $l^i$.

In [ ]:
# Daten filtern (nur Straßen/Wege und Tunnel)
bsr_model = bsr[(bsr['ENTOBJTYP'].isin(['EOT_GWG', 'EOT_TUNANL']))]

# Unbenötigte Attribute entfernen
bsr_model = bsr_model.drop(columns=['BLOCKSEITE', 'STR_VON', 'STR_BIS', 'HOF'])

bsr_model.describe()

In [ ]:
# Berechnung der Länge der Streckenabschnitte (Einheit: m)
bsr_model['l'] = bsr_model['geometry'].length

# Füllen der Nullwerte bei Reinigungsklassen:
bsr_model['RVZRKL'].fillna("NaN", inplace=True)

# Zuordnung der Reinigungshäufigkeiten (Einheit: 1/Woche) zu Reinigungsklassen:
f_map = {
    'A1a': 10,
    'A1b': 7,
    'A2a': 6,
    'A2b': 5,
    'A3': 3,
    'A4': 1,
    'B': 1,
    'C': 1,
    'P': 1, # Modell-Annahme (siehe oben)
    'NaN': 1 # Modell-Annahme (siehe oben)
}

bsr_model['f_pro_Woche'] = bsr_model['RVZRKL']
bsr_model.replace({'f_pro_Woche': f_map}, inplace=True)
bsr_model['f'] = bsr_model['f_pro_Woche'] / 7 # Einheit: 1/Tag

# Unterscheidung nach Besen (f_b) und Kehrmaschinen (f_k)
bsr_model['f_b'] = bsr_model['f'] # Annahme: Alle Gehwege werden mit Besen gereinigt
bsr_model['f_k'] = np.where(bsr_model['RVZRKL'].isin(['C', 'P', 'NaN']), 0, bsr_model['f']) # Annahme: Die von der BSR gereinigten Fahrbahnen werden mit Kehrmaschinen gereinigt, die anderen gar nicht

bsr_model.head()

## Emissionen modellieren
Die Emissionen aus dem Abrieb werden gemäß der Modellbeschreibung aus den vorliegenden Daten bestimmt.

In [ ]:
# Emissionsmengen auf dem Stadtgebiet:
E_b = e_b * p_Berlin
E_k = e_k * p_Berlin

# Proportionalitätskonstante:
alpha_b = 1 / (bsr_model['l'] * bsr_model['f_b']).sum()
alpha_k = 1 / (bsr_model['l'] * bsr_model['f_k']).sum()

# Emissionemengen je Streckenabschnitt:
bsr_model['E_b'] = alpha_b * E_b * bsr_model['l'] * bsr_model['f_b']
bsr_model['E_k'] = alpha_k * E_k * bsr_model['l'] * bsr_model['f_k']

# Streckenspezifische Emissionsmengen je Streckenabschnitt:
bsr_model['epsilon_b'] = alpha_b * E_b  * bsr_model['f_b']
bsr_model['epsilon_k'] = alpha_k * E_k  * bsr_model['f_k']
bsr_model['epsilon'] = bsr_model['epsilon_b'] + bsr_model['epsilon_k']

bsr_model.head()

## Daten visualisieren
### Reinigungshäufigkeit
Die Reinigungshäufigkeit ist nicht nur zur Ermittlung der Abriebsemissionen aus Besen und Kehrmaschinen relevant, sondern auch im Zusammenhang mit der Entfernung von Mikroplastik von der Fahrbahnoberfläche. Daher wird sie hier gesondert visualisiert, bevor die Emissionen dargestellt werden.

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(60,15))
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="2%", pad=0.1)
bsr_model.plot(column='f_pro_Woche', ax= ax, cax=cax, legend = True, legend_kwds={'label': 'Reinigungshäufigkeit [$\\frac{1}{Woche}$]', 'orientation': 'horizontal', 'spacing': 'proportional'})
ctx.add_basemap(ax=ax, crs=bsr_model.crs, source=ctx.providers.Stamen.TonerLite)

### Emissionen aus dem Abrieb von Besen und Kehrmaschinen
In der folgenden Abbildung werden die Modellergebnisse visualisiert. Da die Reinigungshäufigkeit - unter Berücksichtigung der Unterscheidung zwischen Besen und Kehrmaschinen - linear einfließt, ist das Ergebnis visuell nahezu identisch zur Darstellung der Reinigungshäufigkeit.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(60,15))
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="2%", pad=0.1)
bsr_model.plot(column='epsilon', ax= ax, cax=cax, legend = True, legend_kwds={'label': 'Abrieb von Besen und Kehrmaschinen [$\\frac{g}{m \\cdot Jahr}$]', 'orientation': 'horizontal'})
ctx.add_basemap(ax=ax, crs=bsr_model.crs, source=ctx.providers.Stamen.TonerLite)

## Quellen
Amt für Statistik Berlin-Brandenburg (o.J): Bevölkerungsstand. https://www.statistik-berlin-brandenburg.de/bevoelkerung/demografie/bevoelkerungsstand

Berlin Open Data (2015): BSR Straßenreinigung - Verzeichnisse und Reinigungsklassen. https://daten.berlin.de/datensaetze/bsr-stra%C3%9Fenreinigung-verzeichnisse-und-reinigungsklassen

Berlin Open Data (o.J.): Straßenreinigung Berlin. https://daten.berlin.de/anwendungen/stra%C3%9Fenreinigung-berlin

Bertling, Jürgen; Bertling, Ralf; Hamann, Leandra (2018): Kunststoffe in der Umwelt: Mikro- und Makroplastik. Ursachen, Mengen, Umweltschicksale, Wirkungen, Lösungsansätze, Empfehlungen. Kurzfassung der Konsortialstudie, Fraunhofer-Instititut für Umwelt-, Sicherheits- und Energietechnik UMSICHT (Hrsg.), Oberhausen.


BSR (o.J.): Straßenereingung. https://www.bsr.de/strassenreinigung-20471.php

GovData (2013): Blockseiten Köln. https://www.govdata.de/daten/-/details/blockseite

Open Data Düsseldorf (2020): WGS, UTM, ETRS - ein paar Informationen zu Geobasisdaten und Georeferenzsystemen. https://opendata.duesseldorf.de/blog/wgs-utm-etrs-ein-paar-informationen-zu-geobasisdaten-und-georeferenzsystemen